Ben Trevett 의 [Faster Sentiment Analysis](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/3%20-%20Faster%20Sentiment%20Analysis.ipynb) 튜토리얼을 한글 데이터셋에 적용해보는 연습이다. 데이터셋은 [네이버 영화 평점 데이터](https://github.com/e9t/nsmc)을 이용한다.

이 튜토리얼에서는 `FastText` 모델을 이용해서 모델을 경량화해보자.

# 전처리

`FastText` 논문의 핵심 아이디어 중 하나는 입력 문장의 마지막에 문장 구성 토큰들의 n-gram을 추가로 도입하는 것이다. 우리는 여기서 bi-gram을 도입하자. 예를 들어 "how are you ?"의 bi-gram은 "how are", "are you" and "you ?"이다.

따라서 여기서 `generate_bigrams` 함수를 도입하여 토큰화된 문장의 뒤에 bi-gram을 추가하자.

In [1]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

예를 들면

In [3]:
x = ['너', '임마', '밥은', '먹고', '다니냐']
n_grams = set(zip(*[x[i:] for i in range(2)]))
n_grams

{('너', '임마'), ('먹고', '다니냐'), ('밥은', '먹고'), ('임마', '밥은')}

In [4]:
generate_bigrams(['너', '임마', '밥은', '먹고', '다니냐'])

['너', '임마', '밥은', '먹고', '다니냐', '임마 밥은', '밥은 먹고', '먹고 다니냐', '너 임마']

torchtext의 `Field`는 `preprocessing` 과정이 있어서 여기에 함수를 전달하면 토크나이징 후 적용된다. 여기에 `generate_bigrams` 함수를 넣자.

우리는 한글 데이터를 다루므로 토크나이저 또한 별도로 지정해야한다. 여기서는 [KoNLPy](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)의 은전한닢 tokenizer를 이용한다. 또한 패딩을 추가한다. 여기서는 RNN 안쓸 거기 때문에 packed padded seq.를 쓸 수 없어서 `include_lengths=True` 또한 넣을 필요가 없다. 

In [5]:
import torch
from torchtext import data
from torchtext import datasets
from konlpy.tag import Mecab
mecab = Mecab()

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = mecab.morphs, preprocessing = generate_bigrams)
LABEL = data.LabelField(dtype = torch.float)

전처리된 네이버 영화 평점 데이터를 불러오고 검증 데이터를 추가한다.

In [6]:
fields = {'text': ('text',TEXT), 'label': ('label',LABEL)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [7]:
train_data, test_data = data.TabularDataset.splits(
                            path = 'data',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)

In [8]:
import random

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

다음으로 단어 벡터는 전처리된 단어 벡터를 받자. 원 튜토리얼에선 `glove.100d`를 쓰지만 이건 한글을 지원하지 않으므로, 여기선 한글을 지원하는 `fasttext.simple.300d` 를 사용하겠다. 그리고 사전훈련된 단어집에 없는 단어는 0으로 처리하는 걸 방지하기 위해 `unk_init = torch.Tensor.normal_` 옵션을 준다. 

In [9]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                max_size = MAX_VOCAB_SIZE,
                vectors = 'fasttext.simple.300d',
                unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [10]:
len(TEXT.vocab)

25002

In [33]:
TEXT.vocab.itos[:5]

['<unk>', '<pad>', '.', '이', '는']

In [11]:
LABEL.vocab.stoi

defaultdict(None, {'0': 0, '1': 1})

In [26]:
vars(train_data.examples[15])

{'text': ['우리',
  '나라',
  '에',
  '이런',
  '영화',
  '가',
  '더',
  '이상',
  '나오',
  '지',
  '않',
  '았',
  '으면',
  '.',
  '..',
  '가 더',
  '나오 지',
  '이런 영화',
  '지 않',
  '않 았',
  '더 이상',
  '나라 에',
  '영화 가',
  '이상 나오',
  '았 으면',
  '으면 .',
  '. ..',
  '에 이런',
  '우리 나라'],
 'label': '0'}

데이터 생성자를 만들자. 한글 데이터에선 오류가 발생해서 아래와 같이 `sort_key = lambda x: len(x.text)` 문장을 먼저 넣어줘야 오류없이 작동한다.

In [84]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

In [29]:
next(iter(train_iterator)).text

tensor([[ 2533,   360,  3591,  ...,   163,   443, 14207],
        [   54,     3,   103,  ...,   530,   109, 14207],
        [ 2647,   186,    15,  ...,   515,   426,   556],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:0')

In [37]:
TEXT.vocab.itos[2533], TEXT.vocab.itos[54], TEXT.vocab.itos[2647]

('케이블', '에서', '밤')

In [38]:
TEXT.vocab.itos[14207], TEXT.vocab.itos[14207], TEXT.vocab.itos[556]

('머니', '머니', '해도')

[sent_len * batch_size] 형태로 이루어져 있다.

# 모델 생성

여기서는 입력 문장을 임베딩 시킨 후 평균을 취한 다음 행렬곱을 취하는 모델을 사용한다. RNN을 사용하지 않기 때문에 파라미터 수가 훨씬 줄어들었다.

<img src = 'https://github.com/bentrevett/pytorch-sentiment-analysis/raw/79bb86abc9e89951a5f8c4a25ca5de6a491a4f5d/assets/sentiment8.png'>

평균은 다음과 같은 방식으로 취하며 `nn.functional.avg_pool2d` 를 사용한다. 이 함수는 2차원 튜플을 인수로 받으며, 입력 데이터의 마지막 2개 차원을 이용하여 평균을 산출한다.

<img src = 'https://github.com/bentrevett/pytorch-sentiment-analysis/raw/79bb86abc9e89951a5f8c4a25ca5de6a491a4f5d/assets/sentiment10.png'>

In [39]:
import torch.nn as nn
import torch.nn.functional as F

사이즈 계산을 위한 함수를 사용하자.

In [40]:
def print_shape(name, data):
    print(f'{name} has shape {data.shape}')

일단 `avg_pool2d` 에 대해 ARABOZA

In [41]:
nn.functional.avg_pool2d?

Docstring:
avg_pool2d(input, kernel_size, stride=None, padding=0, ceil_mode=False, count_include_pad=True, divisor_override=None) -> Tensor

Applies 2D average-pooling operation in :math:`kH \times kW` regions by step size
:math:`sH \times sW` steps. The number of output features is equal to the number of
input planes.

See :class:`~torch.nn.AvgPool2d` for details and output shape.

Args:
    input: input tensor :math:`(\text{minibatch} , \text{in\_channels} , iH , iW)`
    kernel_size: size of the pooling region. Can be a single number or a
      tuple `(kH, kW)`
    stride: stride of the pooling operation. Can be a single number or a
      tuple `(sH, sW)`. Default: :attr:`kernel_size`
    padding: implicit zero paddings on both sides of the input. Can be a
      single number or a tuple `(padH, padW)`. Default: 0
    ceil_mode: when True, will use `ceil` instead of `floor` in the formula
        to compute the output shape. Default: ``False``
    count_include_pad: when True, will i

In [43]:
txt = torch.rand(2,5,10)
txt.shape, F.avg_pool2d(txt, (5,1)).shape
# (5 x 1) 크기의 필터를 옮겨가며 평균을 구한다.

(torch.Size([2, 5, 10]), torch.Size([2, 1, 10]))

In [45]:
txt = torch.tensor(
    [[[1,2,3,4],[4,5,6,7]]], dtype=torch.float
)
print(txt.shape,"\n", txt)

torch.Size([1, 2, 4]) 
 tensor([[[1., 2., 3., 4.],
         [4., 5., 6., 7.]]])


In [47]:
F.avg_pool2d(txt, (2,1)).shape, F.avg_pool2d(txt, (2,1))
# (2 x 1) 필터로 평균을 취함

(torch.Size([1, 1, 4]), tensor([[[2.5000, 3.5000, 4.5000, 5.5000]]]))

In [48]:
F.avg_pool2d(txt, (2,2)).shape, F.avg_pool2d(txt, (2,2))
# (2 x 2) 필터로 평균을 취함

(torch.Size([1, 1, 2]), tensor([[[3., 5.]]]))

`FastText` 모델을 구현하자. 다만 여기서 주의할 점!

* RNN은 [sent_len, batch_size, embedding_dim] 크기의 텐서를 입력으로 받음
* CNN은 [batch_size, sent_len, embedding_dim] 크기의 텐서를 입력으로 받음

In [85]:
class FastText(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        # text = [sent_len, batch_size]
        #print_shape('text', text)
        
        embedded = self.embedding(text)
        #print_shape('embedded', embedded)
        # embedded = [sent_len, batch_size, embedding_dim]
        
        # CNN은 [batch_size, sent_len, embedding_dim] 를 입력으로 받음
        # 따라서 permute 취해줘야 함
        embedded = embedded.permute(1,0,2)
        #print_shape('embedded', embedded)
        # embedded = [batch_size, sent_len, embedding_dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1],1)).squeeze(1)
        #print_shape('pooled', pooled)
        # pooled = [batch_size, embedding_dim]
        
        res = self.fc(pooled)
        #print_shape('res', res)
        # res = [batch_size, output_dim]
        return res

사이즈 계산 확인해보자

In [86]:
#inp = next(iter(train_iterator))
#model(inp.text)

모델 하이퍼파라미터를 설정하고 인스턴스화 하자.

In [87]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

모델의 파라미터 갯수는?

In [88]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'모델의 파라미터 수는 {count_parameters(model):,} 개 입니다.')

모델의 파라미터 수는 7,500,901 개 입니다.


지난 모델에 비해 약 3/4 으로 감소했다는 것을 알 수 있다.

사전 훈련된 단어 벡터를 덮어 쓰자. 먼저 텐서 차원을 비교해보자.

In [89]:
pretrained_weight = TEXT.vocab.vectors
print(pretrained_weight.shape, model.embedding.weight.data.shape)

torch.Size([25002, 300]) torch.Size([25002, 300])


In [90]:
model.embedding.weight.data.copy_(pretrained_weight)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [ 0.2542,  1.2173,  1.8023,  ..., -0.9746,  0.1054, -1.7293],
        [-0.1084, -0.5668,  0.1102,  ..., -0.5685,  1.6376,  0.2508],
        [-0.3535,  1.0225, -1.7970,  ...,  0.0683,  0.3403,  1.5236]])

`UNK_IDX`와 `PAD_IDX`는 제로 처리한다.

In [91]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# 모델 훈련

이전과 동일하게 하자

In [92]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [93]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [94]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

훈련 함수를 정의하자. 여기선 드랍아웃 안쓰지만 걍 `model.train()` 사용하겠다.

In [95]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1) # output_dim = 1
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [96]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

얼마나 훈련 걸리는 지 체크하는 함수

In [97]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

이제 훈련시켜보자.


In [98]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 7s
	Train Loss: 0.442 | Train Acc: 80.87%
	 Val. Loss: 0.360 |  Val. Acc: 85.12%
Epoch: 02 | Epoch Time: 0m 7s
	Train Loss: 0.315 | Train Acc: 87.39%
	 Val. Loss: 0.342 |  Val. Acc: 86.13%
Epoch: 03 | Epoch Time: 0m 7s
	Train Loss: 0.277 | Train Acc: 89.06%
	 Val. Loss: 0.345 |  Val. Acc: 86.28%
Epoch: 04 | Epoch Time: 0m 7s
	Train Loss: 0.254 | Train Acc: 90.07%
	 Val. Loss: 0.354 |  Val. Acc: 86.31%
Epoch: 05 | Epoch Time: 0m 7s
	Train Loss: 0.238 | Train Acc: 90.66%
	 Val. Loss: 0.368 |  Val. Acc: 85.94%


나쁘지 않군! 게다가 훈련 시간은 1/4 이다.

테스트셋에서 돌려보자.

In [99]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.345 | Test Acc: 86.18%


더 훈련시켜보자.

In [100]:
for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+6:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 06 | Epoch Time: 0m 7s
	Train Loss: 0.275 | Train Acc: 89.17%
	 Val. Loss: 0.345 |  Val. Acc: 86.28%
Epoch: 07 | Epoch Time: 0m 7s
	Train Loss: 0.253 | Train Acc: 90.11%
	 Val. Loss: 0.356 |  Val. Acc: 86.16%
Epoch: 08 | Epoch Time: 0m 7s
	Train Loss: 0.238 | Train Acc: 90.68%
	 Val. Loss: 0.368 |  Val. Acc: 85.99%
Epoch: 09 | Epoch Time: 0m 7s
	Train Loss: 0.226 | Train Acc: 91.16%
	 Val. Loss: 0.385 |  Val. Acc: 85.60%
Epoch: 10 | Epoch Time: 0m 7s
	Train Loss: 0.216 | Train Acc: 91.56%
	 Val. Loss: 0.400 |  Val. Acc: 85.50%


오버피팅이 발생하고 있다...

In [101]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.345 | Test Acc: 86.18%


성능은 이전 모델과 거의 비슷하지만 훈련 시간이 대폭 감소!

# 사용자 데이터 사용

영화 평가 데이터 직접 넣어보자.

다음 기능을 하는 `predict_sentiment` 함수를 만들자.

* sets the model to evaluation mode
* tokenizes the sentence, i.e. splits it from a raw string into a list of tokens
* indexes the tokens by converting them into their integer representation from our vocabulary
* gets the length of our sequence
* converts the indexes, which are a Python list into a PyTorch tensor
* add a batch dimension by unsqueezeing
* converts the length into a tensor
* squashes the output prediction from a real number between 0 and 1 with the `sigmoid` function
* converts the tensor holding a single value into an integer with the item() method

In [102]:
from konlpy.tag import Mecab
mecab = Mecab()

In [103]:
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok for tok in mecab.morphs(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1) # 배치 
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [104]:
predict_sentiment(model, "이 영화 진짜 재밌었다!!")

0.9986088871955872

In [105]:
predict_sentiment(model, "영화관에서 이걸 본 내가 바보다. 내 돈 돌려줘!!!")

0.0041242544539272785

In [106]:
predict_sentiment(model, "이 영화 감독 밥은 먹고 다니냐? 이런 영화 만들고 잠이 와?")

0.020292015746235847

In [107]:
predict_sentiment(model, "내 인생 영화 등극. 주인공한테 너무 몰입해서 시간 가는 줄도 몰랐다...")

0.9285458922386169

Not bad!